In [1]:
import requests
from pathlib import Path
from datetime import datetime, timedelta
import pickle
import logging

from utils import *
from tqdm.auto import tqdm

In [2]:
from meteostat import Point, Daily
from datetime import datetime

# Set time period
start = datetime(2015, 1, 1)
end = datetime(2023, 12, 31)

# Create Point for Vancouver, BC
location = Point(12.011616, 79.811897)

# Get daily data for 2018
data = Daily(location, start, end)
data = data.fetch()
data

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2015-01-01,25.4,23.5,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,26.5,23.1,31.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-03,26.6,23.6,31.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-04,26.2,23.1,30.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,25.9,22.3,30.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-12-27,25.2,22.1,30.0,0.0,NaN,11.0,27.0,NaN,1015.4,NaN
2023-12-28,26.0,23.6,31.0,1.7,NaN,28.0,27.2,NaN,1014.4,NaN
2023-12-29,25.6,23.6,30.0,2.7,NaN,31.0,24.8,NaN,1014.6,NaN


In [2]:
logger = logging.getLogger("my_custom_logger")
handler = logging.FileHandler("data_download.log")


logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger.addHandler(handler)


pricearrival = agmarknetOptions.price_arrival.Both
state = agmarknetOptions.state.__Select__


crop_name = dir(agmarknetOptions.commodities)
fol_path = Path('TN')


for crop in crop_name:
    
    commodity = eval(f'agmarknetOptions.commodities.{crop}')
    file_path = fol_path / f"{commodity[0].replace('/', '-')}.feather"
    from_dt = dt.date(2010, 1, 1)
    to_dt = dt.date(2024, 4, 11)

    if file_path.is_file():
        continue

    print(crop)
    try:

        df = []

        while from_dt < to_dt:
            cur_to_date = min(from_dt.replace(year=from_dt.year + 2), to_dt)
            print(from_dt, cur_to_date)
            response = agmarknet(from_dt.strftime('%Y-%m-%d'), cur_to_date.strftime('%Y-%m-%d'), pricearrival, commodity, state)
            
            if df:
                df += response[1:]
            else:
                df += response


            from_dt = cur_to_date + dt.timedelta(days=1)

        if df:
            df = pd.DataFrame(df[1:], columns=df[0])
            df.to_feather(file_path)
            logger.info(f'{crop}, {df.shape[0]}')
        else:
            with open(file_path, 'wb') as f:
                f.write("")
    
    except Exception as e:
        with open(file_path, 'wb') as f:
                f.write("error")
        logging.error(f'{crop}, {e}')

Amaranthus
2010-01-01 2012-01-01
2012-01-02 2014-01-02
2014-01-03 2016-01-03
2016-01-04 2018-01-04
2018-01-05 2020-01-05
2020-01-06 2022-01-06
2022-01-07 2024-01-07
2024-01-08 2024-04-11
Amphophalus
2010-01-01 2012-01-01
2012-01-02 2014-01-02
2014-01-03 2016-01-03
2016-01-04 2018-01-04
2018-01-05 2020-01-05
2020-01-06 2022-01-06
2022-01-07 2024-01-07
2024-01-08 2024-04-11
Apple
2010-01-01 2012-01-01


TypeError: a bytes-like object is required, not 'str'